Trabalho realizado por: Bárbara Freixo, PG49169


Esta implementação implementa uma classe que define a lógica para o algoritmo de classificação Prism. Ela possui métodos para inicializar os atributos, encontrar as regras de classificação, avaliar uma regra, verificar se uma regra cobre um dado, retornar a classe majoritária num conjunto de dados, realizar a classificação de novos dados e representar a classe como string. O método "fit" é o responsável por encontrar as regras de classificação, ele funciona enquanto houver dados para serem classificados, procurando a melhor regra em cada iteração e adicionando-a à lista de regras. O método "predict" é o responsável por realizar a classificação de novos dados, ele percorre cada dado e aplica as regras encontradas na iteração do método "fit" até encontrar uma regra que cobre o dado, caso nenhuma regra cobra o dado, a classe majoritária dos dados iniciais é retornada.
        

In [8]:
import numpy as np

class Prism:
    def __init__(self, data, target, attributes):
        """
        Inicializa o objeto Prism com o conjunto de dados, a coluna alvo e os atributos.
        
        :param data: lista de listas contendo os dados de treino
        :param target: int, índice da coluna alvo no conjunto de dados
        :param attributes: lista de int, índices das colunas de atributos no conjunto de dados
        """
        self.data = data
        self.target = target
        self.attributes = attributes
        self.rules = []
        self.default_class = self.majority_class(data)

    def fit(self):
        """
        Encontra as regras que melhor separam as classes no conjunto de dados.
        """
        remaining_data = self.data.copy()
        while len(remaining_data) > 0:
            best_rule = None
            best_accuracy = 0

            for attribute in self.attributes:
                for value in set([row[attribute] for row in remaining_data]):
                    rule = (attribute, value)
                    accuracy = self.evaluate_rule(rule, remaining_data)

                    if accuracy > best_accuracy:
                        best_rule = rule
                        best_accuracy = accuracy

            if best_rule is None:
                break

            covered_data = [row for row in remaining_data if self.rule_covers(row, best_rule)]
            rule_class = self.majority_class(covered_data)
            self.rules.append((best_rule, rule_class))
            remaining_data = [row for row in remaining_data if not self.rule_covers(row, best_rule)]

    def evaluate_rule(self, rule, data):
        """
        Avalia a acurácia de uma regra no conjunto de dados fornecido.
        
        :param rule: tupla (attribute, value), representa a regra
        :param data: lista de listas contendo os dados a serem avaliados
        :return: float, acurácia da regra nos dados fornecidos
        """
        covered = [row for row in data if self.rule_covers(row, rule)]
        if len(covered) == 0:
            return 0
        correct_covered = [row for row in covered if row[self.target] == self.majority_class(covered)]
        return len(correct_covered) / len(covered)

    def rule_covers(self, row, rule):
        """
        Verifica se uma regra cobre uma instância de dados.
        
        :param row: lista, representa uma instância de dados
        :param rule: tupla (attribute, value), representa a regra
        :return: bool, True se a regra cobrir a instância de dados, False caso contrário
        """
        attribute, value = rule
        return row[attribute] == value

    def majority_class(self, data):
        """
        Encontra a classe majoritária no conjunto de dados fornecido.
        
        :param data: lista de listas contendo os dados a serem avaliados
        :return: valor da classe majoritária nos dados fornecidos
        """
        if len(data) == 0:
            return self.default_class
        classes = [row[self.target] for row in data]
        return max(set(classes), key=classes.count)

    def predict(self, data):
        """
        Faz previsões para um conjunto de dados usando as regras encontradas.
        
        :param data: lista de listas contendo os dados a serem classificados
        :return: lista de preções para cada instância de dados no conjunto de dados fornecido
        """
        predictions = []
        for row in data:
            for rule, target_class in self.rules:
                if self.rule_covers(row, rule):
                    predictions.append(target_class)
                    break
            else:
                predictions.append(self.majority_class(self.data))
        return predictions

    def __repr__(self):
        """
        Representação de string do objeto Prism, mostrando as regras encontradas.
        
        :return: string, representação das regras encontradas
        """
        rule_strings = [f"IF {attribute} == {value} THEN class = {target_class}" for (attribute, value), target_class in self.rules]
        return "\n".join(rule_strings)

# Testes e exemplos para o algoritmo implementado

## Exemplo de uso do código

Esta implementação é um exemplo de como usar o classificador Prism num dataset. Ele carrega o dataset Iris, divide os dados em conjuntos de treino e teste, treina o classificador Prism com os dados de treino e preve as classes nos dados de teste. Finalmente, ele calcula a precisão do classificador usando a função accuracy_score da biblioteca sklearn. O resultado é exibido na tela com as regras do classificador e a precisão.

In [11]:
from sklearn import datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Carrega o dataset Iris
iris = datasets.load_iris()

# Transforma os dados numa lista de dicionários, onde cada chave é o nome do atributo e o valor é o valor do atributo
data = [
    {attribute: value for attribute, value in zip(iris.feature_names, row)}
    for row in iris.data
]
target = 'class'

# Adiciona a coluna alvo (classe) aos dados
for row, target_value in zip(data, iris.target):
    row[target] = target_value

# Discretiza atributos contínuos usando 'n_bins' como número de intervalos
n_bins = 5
for attribute in iris.feature_names:
    attribute_values = [row[attribute] for row in data]
    bins = pd.cut(attribute_values, bins=n_bins, labels=list(range(n_bins)))
    for row, bin_value in zip(data, bins):
        row[attribute] = bin_value

# Divide os dados em conjuntos de treinamento e teste
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Remove a coluna alvo do conjunto de teste para fazer as previsões
test_data_no_target = [{k: v for k, v in row.items() if k != target} for row in test_data]

# Cria e treina o classificador Prism
attributes = iris.feature_names
prism = Prism(train_data, target, attributes)
prism.fit()

# Realiza previsões no conjunto de teste
predictions = prism.predict(test_data_no_target)

# Calcula a acurácia das previsões
y_true = [row[target] for row in test_data]
accuracy = accuracy_score(y_true, predictions)

# Exibe as regras encontradas pelo classificador Prism e a acurácia
print("Prism classifier rules:")
print(prism)
print(f"\nAccuracy: {accuracy:.2f}")

Prism classifier rules:
IF sepal length (cm) == 4 THEN class = 2
IF sepal width (cm) == 3 THEN class = 0
IF sepal width (cm) == 4 THEN class = 0
IF petal length (cm) == 0 THEN class = 0
IF petal length (cm) == 1 THEN class = 1
IF petal length (cm) == 4 THEN class = 2
IF petal width (cm) == 1 THEN class = 1
IF sepal length (cm) == 0 THEN class = 2
IF petal length (cm) == 2 THEN class = 1
IF sepal length (cm) == 1 THEN class = 2
IF sepal width (cm) == 0 THEN class = 2
IF petal width (cm) == 4 THEN class = 2
IF petal width (cm) == 2 THEN class = 1
IF sepal width (cm) == 1 THEN class = 2
IF sepal length (cm) == 2 THEN class = 2
IF sepal length (cm) == 3 THEN class = 2

Accuracy: 0.97


## Testes "Unittest"

Foi realizado um teste utilizando o unittest para testar a implementação do PRISM. Temos então o seguinte teste:

In [10]:
import unittest

class TestPrism(unittest.TestCase):
    #Test the functionality of the Prism classifier
    def test_prism_classifier(self):
        data = [
            {'outlook': 'sunny', 'temperature': 'hot', 'humidity': 'high', 'wind': 'weak', 'class': 'no'},
            {'outlook': 'sunny', 'temperature': 'hot', 'humidity': 'high', 'wind': 'strong', 'class': 'no'},
            {'outlook': 'overcast', 'temperature': 'hot', 'humidity': 'high', 'wind': 'weak', 'class': 'yes'},
            {'outlook': 'rain', 'temperature': 'mild', 'humidity': 'high', 'wind': 'weak', 'class': 'yes'},
            {'outlook': 'rain', 'temperature': 'cool', 'humidity': 'normal', 'wind': 'weak', 'class': 'yes'},
            {'outlook': 'rain', 'temperature': 'cool', 'humidity': 'normal', 'wind': 'strong', 'class': 'no'},
            {'outlook': 'overcast', 'temperature': 'cool', 'humidity': 'normal', 'wind': 'strong', 'class': 'yes'},
            {'outlook': 'sunny', 'temperature': 'mild', 'humidity': 'high', 'wind': 'weak', 'class': 'no'},
            {'outlook': 'sunny', 'temperature': 'cool', 'humidity': 'normal', 'wind': 'weak', 'class': 'yes'},
            {'outlook': 'rain', 'temperature': 'mild', 'humidity': 'normal', 'wind': 'weak', 'class': 'yes'},
            {'outlook': 'sunny', 'temperature': 'mild', 'humidity': 'normal', 'wind': 'strong', 'class': 'yes'},
            {'outlook': 'overcast', 'temperature': 'mild', 'humidity': 'high', 'wind': 'strong', 'class': 'yes'},
            {'outlook': 'overcast', 'temperature': 'hot', 'humidity': 'normal', 'wind': 'weak', 'class': 'yes'},
            {'outlook': 'rain', 'temperature': 'mild', 'humidity': 'high', 'wind': 'strong', 'class': 'no'},
        ]
        attributes = ['outlook', 'temperature', 'humidity', 'wind']
        target = 'class'
        prism = Prism(data, target, attributes)

        # Test rule creation
        prism.fit()
        self.assertGreater(len(prism.rules), 0, "No rules were created.")

        # Test prediction
        test_data = [{'outlook': 'sunny', 'temperature': 'cool', 'humidity': 'normal', 'wind': 'weak'}]
        predictions = prism.predict(test_data)
        expected_predictions = ['yes']
        self.assertEqual(predictions, expected_predictions, "Incorrect predictions.")

        # Test accuracy
        data_no_target = [{k: v for k, v in row.items() if k != target} for row in data]
        predictions = prism.predict(data_no_target)
        y_true = [row[target] for row in data]
        accuracy = accuracy_score(y_true, predictions)
        self.assertGreater(accuracy, 0, "Incorrect accuracy calculation.")

        # Test __repr__ method
        repr_result = repr(prism)
        self.assertIsInstance(repr_result, str, "Incorrect __repr__ result.")

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


Esta implementação testa o funcionamento do classificador Prism. Ela cria uma instância do classificador Prism usando um conjunto de dados de exemplo e conjuntos de atributos e variáveis alvo. A implementação então testa a criação de regras, previsões, precisão e a representação em string do classificador.